In [2]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Chargement du jeu de données Iris
iris = load_iris()
X = iris.data
y = iris.target

# Séparation des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialisation et apprentissage du modèle k-NN
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

# Prédiction sur les données de test
y_pred = knn.predict(X_test)

# Calcul de l'exactitude du modèle
accuracy = accuracy_score(y_test, y_pred)
print("Exactitude du modèle k-NN : {:.2f}".format(accuracy))


Exactitude du modèle k-NN : 1.00


L'algorithme d'inférence variationnelle est une méthode d'approximation utilisée pour estimer les distributions postérieures dans les modèles probabilistes.Cette approche permet de simplifier le problème d'estimation en remplaçant la distribution postérieure difficile à calculer par une distribution plus facile à manipuler.
L'objectif est de trouver les paramètres de cette distribution approximative qui minimisent la divergence entre la distribution approximative et la distribution postérieure réelle.

L'inférence variationnelle est une approche couramment utilisée lorsque la distribution postérieure exacte est inconnue ou intractable, et elle offre une alternative efficace aux méthodes d'échantillonnage plus coûteuses en termes de calcul, telles que l'échantillonnage de Monte Carlo par chaînes de Markov (MCMC).

principe:

1-Modèle probabiliste : Tout d'abord, on définit un modèle probabiliste qui décrit la relation entre les variables observées et les variables latentes inconnues. Ce modèle est généralement basé sur des hypothèses probabilistes et peut prendre la forme d'un modèle graphique, d'un modèle bayésien, etc.

2-Distribution postérieure : L'objectif est d'estimer la distribution postérieure des variables latentes étant donné les observations. Cependant, calculer exactement cette distribution peut être difficile, voire impossible, en raison de l'intractabilité mathématique ou de la complexité du modèle.

3-Distribution approximative : On choisit une distribution approximative appartenant à une famille paramétrique plus restreinte qui sera plus facile à manipuler. Cette distribution est généralement sélectionnée en fonction de sa flexibilité et de sa capacité à s'adapter au modèle.

4-Minimisation de la divergence de KL : L'objectif est de trouver les paramètres de la distribution approximative qui minimisent la divergence de KL entre la distribution approximative et la distribution postérieure réelle. Cela revient à trouver les meilleurs paramètres qui "rapprochent" la distribution approximative de la distribution postérieure.

5-Algorithme d'optimisation : Pour minimiser la divergence de KL, on utilise des techniques d'optimisation numérique telles que la descente de gradient, l'optimisation par coordonnées ou l'optimisation stochastique. L'algorithme d'optimisation ajuste itérativement les paramètres de la distribution approximative jusqu'à ce qu'une convergence satisfaisante soit atteinte.

6-Estimation de la distribution postérieure : Une fois que les paramètres de la distribution approximative ont été optimisés, on utilise cette distribution approximative pour estimer la distribution postérieure des variables latentes. Cette estimation est une approximation de la distribution postérieure réelle, mais elle peut être utilisée pour effectuer des prédictions, des analyses statistiques ou d'autres tâches d'inférence.

.


In [1]:
import numpy as np
import pymc3 as pm
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Charger le jeu de données Iris
iris = load_iris()
X = iris.data
y = iris.target

# Prétraitement des étiquettes en encodage entier
le = LabelEncoder()
y = le.fit_transform(y)

# Fractionnement des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def bayesian_knn(x_train, y_train, x_test, k):
    n_train = x_train.shape[0]
    n_test = x_test.shape[0]

    with pm.Model() as model:
        # Prior pour les classes
        p = pm.Dirichlet('p', a=np.ones(len(np.unique(y_train))), shape=len(np.unique(y_train)))

        # Calcul des distances entre les échantillons d'entraînement et les échantillons de test
        dists = np.sum((x_train[:, np.newaxis] - x_test) ** 2, axis=2)

        # Extraction des indices des k plus petites distances pour chaque échantillon de test
        k_indices = np.argpartition(dists, k, axis=0)[:k]

        # Extraction des étiquettes correspondantes aux indices des k plus petites distances
        k_labels = y_train[k_indices]

        # Définition des variables catégorielles likelihood pour chaque échantillon de test
        for i in range(n_test):
            likelihood = pm.Categorical(f'likelihood_{i}', p=p[k_labels[:, i]], observed=k_labels[:, i])

        # Inférence variationnelle
        approx = pm.fit(n=500, method='advi')

        # Échantillonnage à partir de la distribution approximative
        trace = approx.sample(500)

    # Réduire la dimension de y_pred_bayesian pour avoir la même longueur que y_test
    y_pred_bayesian = np.argmax(trace['p'], axis=1)[:len(y_test)]

    # Calcul de la précision du k-NN bayésien
    accuracy_bayesian = accuracy_score(y_test, y_pred_bayesian)

    return model, approx, accuracy_bayesian

# Utilisation de l'algorithme de k-NN bayésien avec inférence variationnelle sur les données d'entraînement et de test
model, approx, accuracy = bayesian_knn(X_train, y_train, X_test, k=3)

print("Précision du k-NN bayésien :", accuracy)


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Précision du k-NN bayésien : 0.3333333333333333
